In [1]:
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import sys
from IPython.display import HTML
from matplotlib import pyplot as plt
from celluloid import Camera
import plotly.graph_objects as go

import networkx as nx

from IPython.display import HTML
from matplotlib import pyplot as plt
from celluloid import Camera
import collections


import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
import ast

In [2]:
emails_df = pd.read_csv('Data/emails_tokenized.csv')
employee_df = pd.read_csv('Data/employees.csv')
network_df = pd.read_csv('Data/network.csv')

In [3]:
emails_df = emails_df[~emails_df['token_subject'].isna()]

exec_list = ['CEO','President','Vice President']

job_count = collections.Counter()
for rank in emails_df['sender_level']:
    job_count[rank] +=1

job_count

In [6]:
stems = ['fw','draft','p','2001']

emails_df['token_subject']= [ast.literal_eval(x) for x in emails_df['token_subject']]
subjects = []
for email in emails_df['token_subject']:
    for stem in stems:
        while stem in email:
            email.remove(stem)
    subjects.append(email)
emails_df['token_subject'] = subjects

In [32]:
emails_df.head()

,Unnamed: 0,mid,sender,date,message_id,subject,body,folder,cleaned_body,cleaned_subject,sender_level,token_body,token_subject
0,24,76,mary.hain@enron.com,2000-08-17 07:11:00,<5722839.1075863606028.JavaMail.evans@thyme>,Enron s transmission/power exchange model for ...,---------------------- Forwarded by Mary Hain/...,Robert_Badeer_Aug2000Notes FoldersNotes inbox,enron s transmission/power exchange model for...,enron s transmission/power exchange model for ...,NaN,"[[[enron, transmission/power, exchange, model,...","[enron, transmission/power, exchange, model, d..."
1,41,93,cooper.richey@enron.com,2000-08-23 04:39:00,<4083763.1075863605647.JavaMail.evans@thyme>,Change to EnData,The Fundamentals Group is moving Database serv...,Robert_Badeer_Aug2000Notes FoldersNotes inbox,the fundamentals group is moving database serv...,change to endata,Manager,"[[[fundamentals, group, moving, database, serv...","[change, endata]"
2,107,159,mary.hain@enron.com,2000-08-29 06:28:00,<14457348.1075863603979.JavaMail.evans@thyme>,ISO To Participate in Super Peak Market,FYI---------------------- Forwarded by Mary Ha...,Robert_Badeer_Aug2000Notes FoldersNotes inbox,"iso to participate in super peak marketfolks,...",iso to participate in super peak et,NaN,"[[[iso, participate, super, peak, marketfolks,...","[iso, participate, super, peak, et]"
3,137,465,jeff.dasovich@enron.com,2000-08-07 12:23:00,<769016.1075863596202.JavaMail.evans@thyme>,CalPX Market Compliance Unit Submits its 2nd A...,FYI. The report posted on the website fails t...,Robert_Badeer_Aug2000Notes FoldersCapx,calpx market compliance unit submits its 2nd ...,calpx et compliance unit submits its 2nd annu...,Employee,"[[[calpx, market, compliance, unit, submits, 2...","[calpx, et, compliance, unit, submits, 2nd, an..."
4,189,517,jeff.dasovich@enron.com,2000-08-10 03:12:00,<28494229.1075863594884.JavaMail.evans@thyme>,Re: Governor s Press Conference -- Shot Across...,There appears to be a pattern forming. The ad...,Robert_Badeer_Aug2000Notes FoldersCalifornia,governor s press conference -- shot across th...,re: governor s press conference -- shot across...,Employee,"[[[governor, press, conference, --, shot, acro...","[governor, press, conference, --, shot, across..."


In [8]:
emails_df['token_body']= [ast.literal_eval(x) for x in emails_df['token_body']]
subjects = []
for email in emails_df['token_body']:
    sents = []
    for sent in email:
        for stem in stems:
            while stem in sent:
                sent.remove(stem)
        sents.append(email)
    subjects.append(sents)
emails_df['token_body'] = subjects

In [9]:
subject_tokens = list(emails_df['token_subject']) 

In [10]:
q

In [11]:
dictionary = gensim.corpora.Dictionary(subject_tokens)

In [12]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 discussion
1 enron
2 exchange
3 model
4 transmission/power
5 change
6 endata
7 et
8 iso
9 participate
10 peak


In [13]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [14]:
bow_corpus = [dictionary.doc2bow(doc) for doc in subject_tokens]
bow_corpus[4310]

[(421, 1), (474, 1), (820, 1)]

In [15]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5546227871296954),
 (1, 0.3020901308523857),
 (2, 0.5623506333794593),
 (3, 0.5337573249856136)]


In [16]:
topic_num = 5
show_feature = 7

In [17]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=topic_num, id2word=dictionary, passes=20, workers=2)

Topic: 0 
Words: 0.053*"enron" + 0.035*"energy" + 0.023*"letter" + 0.020*"eol" + 0.019*"trading" + 0.018*"agreement" + 0.017*"credit" + 0.017*"``" + 0.016*"''" + 0.016*"isda"
Topic: 1 
Words: 0.056*"gas" + 0.035*"deal" + 0.031*"agreement" + 0.021*"california" + 0.018*"list" + 0.017*"capacity" + 0.017*"request" + 0.013*"revised" + 0.012*"info" + 0.011*"e"
Topic: 2 
Words: 0.064*"meeting" + 0.025*"..." + 0.022*"call" + 0.019*"conference" + 0.011*"game" + 0.010*"management" + 0.010*"april" + 0.010*"issues" + 0.010*"today" + 0.010*"friday"
Topic: 3 
Words: 0.056*"new" + 0.027*"power" + 0.017*"deals" + 0.016*"schedule" + 0.012*"information" + 0.011*"book" + 0.011*"plan" + 0.011*"storage" + 0.010*"west" + 0.010*"netco"
Topic: 4 
Words: 0.033*"update" + 0.025*"contract" + 0.020*"--" + 0.018*"report" + 0.016*"please" + 0.014*"lunch" + 0.013*"dinner" + 0.012*"final" + 0.012*"ca" + 0.011*"presentation"


In [19]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=topic_num, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.028*"deal" + 0.024*"meeting" + 0.012*"hello" + 0.009*"netco" + 0.009*"information" + 0.008*"enron" + 0.008*"isda" + 0.007*"good" + 0.007*"gas" + 0.007*"document"
Topic: 1 Word: 0.021*"agreement" + 0.016*"call" + 0.016*"contract" + 0.014*"hey" + 0.012*"conference" + 0.011*"tw" + 0.009*"contracts" + 0.009*"book" + 0.009*"approval" + 0.008*"meeting"
Topic: 2 Word: 0.019*"power" + 0.015*"gas" + 0.015*"master" + 0.013*"energy" + 0.013*"agreement" + 0.012*"info" + 0.010*"project" + 0.009*"inc" + 0.009*"ces" + 0.008*"new"
Topic: 3 Word: 0.016*"lunch" + 0.015*"update" + 0.014*"..." + 0.012*"gisb" + 0.011*"letter" + 0.010*"checkout" + 0.010*"ca" + 0.009*"test" + 0.009*"trading" + 0.008*"e"
Topic: 4 Word: 0.016*"new" + 0.014*"enron" + 0.013*"credit" + 0.012*"california" + 0.011*"eol" + 0.011*"hi" + 0.010*"fwd" + 0.009*"capacity" + 0.009*"game" + 0.009*"resume"


In [20]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, show_feature)))


Score: 0.5500427484512329	 
Topic: 0.053*"enron" + 0.035*"energy" + 0.023*"letter" + 0.020*"eol" + 0.019*"trading" + 0.018*"agreement" + 0.017*"credit"

Score: 0.2992553114891052	 
Topic: 0.064*"meeting" + 0.025*"..." + 0.022*"call" + 0.019*"conference" + 0.011*"game" + 0.010*"management" + 0.010*"april"

Score: 0.05069613456726074	 
Topic: 0.056*"gas" + 0.035*"deal" + 0.031*"agreement" + 0.021*"california" + 0.018*"list" + 0.017*"capacity" + 0.017*"request"

Score: 0.05000292509794235	 
Topic: 0.033*"update" + 0.025*"contract" + 0.020*"--" + 0.018*"report" + 0.016*"please" + 0.014*"lunch" + 0.013*"dinner"

Score: 0.05000290274620056	 
Topic: 0.056*"new" + 0.027*"power" + 0.017*"deals" + 0.016*"schedule" + 0.012*"information" + 0.011*"book" + 0.011*"plan"


In [21]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, show_feature)))


Score: 0.4719347357749939	 
Topic: 0.016*"new" + 0.014*"enron" + 0.013*"credit" + 0.012*"california" + 0.011*"eol" + 0.011*"hi" + 0.010*"fwd"

Score: 0.3750551640987396	 
Topic: 0.019*"power" + 0.015*"gas" + 0.015*"master" + 0.013*"energy" + 0.013*"agreement" + 0.012*"info" + 0.010*"project"

Score: 0.052007295191287994	 
Topic: 0.016*"lunch" + 0.015*"update" + 0.014*"..." + 0.012*"gisb" + 0.011*"letter" + 0.010*"checkout" + 0.010*"ca"

Score: 0.050592586398124695	 
Topic: 0.028*"deal" + 0.024*"meeting" + 0.012*"hello" + 0.009*"netco" + 0.009*"information" + 0.008*"enron" + 0.008*"isda"

Score: 0.05041024833917618	 
Topic: 0.021*"agreement" + 0.016*"call" + 0.016*"contract" + 0.014*"hey" + 0.012*"conference" + 0.011*"tw" + 0.009*"contracts"


In [22]:
unseen_document = ['how','pentagon','be']
bow_vector = dictionary.doc2bow(subject_tokens[0])
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, show_feature)))

Score: 0.44001737236976624	 Topic: 0.053*"enron" + 0.035*"energy" + 0.023*"letter" + 0.020*"eol" + 0.019*"trading" + 0.018*"agreement" + 0.017*"credit"
Score: 0.43975332379341125	 Topic: 0.056*"gas" + 0.035*"deal" + 0.031*"agreement" + 0.021*"california" + 0.018*"list" + 0.017*"capacity" + 0.017*"request"
Score: 0.04021953418850899	 Topic: 0.064*"meeting" + 0.025*"..." + 0.022*"call" + 0.019*"conference" + 0.011*"game" + 0.010*"management" + 0.010*"april"
Score: 0.04000651091337204	 Topic: 0.056*"new" + 0.027*"power" + 0.017*"deals" + 0.016*"schedule" + 0.012*"information" + 0.011*"book" + 0.011*"plan"
Score: 0.04000326991081238	 Topic: 0.033*"update" + 0.025*"contract" + 0.020*"--" + 0.018*"report" + 0.016*"please" + 0.014*"lunch" + 0.013*"dinner"


array([[5.4903303e-06, 5.3315543e-02, 2.4928332e-03, ..., 5.6026879e-06,
        5.6026906e-06, 1.0235189e-03],
       [1.7291348e-03, 5.3949725e-06, 5.4043680e-06, ..., 1.3964525e-03,
        5.3436179e-06, 5.3944814e-06],
       [6.0666073e-04, 5.1074662e-06, 5.0873814e-06, ..., 5.2075720e-06,
        5.8230848e-06, 5.0471499e-06],
       [5.2735081e-06, 3.0947936e-04, 5.3377285e-06, ..., 5.5158421e-06,
        5.3875215e-06, 5.5603550e-06],
       [6.3606317e-06, 6.3603766e-06, 6.4196256e-06, ..., 6.5366889e-06,
        1.2319734e-03, 6.5161471e-06]], dtype=float32)

In [23]:
subject_topics = []
for subject in emails_df['token_subject']:
    bow_vector = dictionary.doc2bow(subject)
    subject_topics.append(dict(lda_model[bow_vector]))

In [30]:
lda_model.get_topic_terms(0, topn=10)

[(1, 0.053315543),
 (37, 0.034853052),
 (121, 0.022558168),
 (141, 0.020312788),
 (421, 0.019311314),
 (88, 0.01754928),
 (117, 0.016980043),
 (266, 0.016740583),
 (208, 0.016245471),
 (861, 0.015849246)]

array([[5.4903303e-06, 5.3315543e-02, 2.4928332e-03, ..., 5.6026879e-06,
        5.6026906e-06, 1.0235189e-03],
       [1.7291348e-03, 5.3949725e-06, 5.4043680e-06, ..., 1.3964525e-03,
        5.3436179e-06, 5.3944814e-06],
       [6.0666073e-04, 5.1074662e-06, 5.0873814e-06, ..., 5.2075720e-06,
        5.8230848e-06, 5.0471499e-06],
       [5.2735081e-06, 3.0947936e-04, 5.3377285e-06, ..., 5.5158421e-06,
        5.3875215e-06, 5.5603550e-06],
       [6.3606317e-06, 6.3603766e-06, 6.4196256e-06, ..., 6.5366889e-06,
        1.2319734e-03, 6.5161471e-06]], dtype=float32)

In [53]:
exec_email_df = emails_df[[x in exec_list for x in emails_df['sender_level']]]
employee_email_df = emails_df[emails_df['sender_level']=='Employee']

Total_Emails = list(employee_email_df['token_subject'])+list(exec_email_df['token_subject'])

exec_dictionary = gensim.corpora.Dictionary(Total_Emails)
exec_dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

exec_corpus = [dictionary.doc2bow(doc) for doc in list(exec_email_df['token_subject'])]

employee_corpus =[dictionary.doc2bow(doc) for doc in list(employee_email_df['token_subject'])]


Exec_model = gensim.models.LdaMulticore(exec_corpus, num_topics=topic_num, id2word=dictionary, passes=20, workers=2)
Employee_model = gensim.models.LdaMulticore(employee_corpus, num_topics=topic_num, id2word=dictionary, passes=20, workers=2)

In [56]:
mdiff, annotation = Exec_model.diff(Employee_model)

In [57]:
mdiff

array([[0.97538688, 0.96850545, 0.90484161, 0.6768113 , 0.91820328],
       [0.89332831, 0.77634917, 0.7928334 , 0.90771066, 1.        ],
       [0.787698  , 0.85637377, 0.86991786, 0.8691997 , 0.93276679],
       [0.78568009, 0.82593486, 0.94245981, 0.92296012, 0.76712376],
       [0.85035395, 0.88523307, 0.86799871, 0.90570744, 0.64555964]])

In [64]:
Exec_model.top_topics(employee_corpus,topn=5)

[([(0.024912834, 'power'),
   (0.023507157, 'call'),
   (0.019813964, '--'),
   (0.014668309, 'conference'),
   (0.014410517, 'issues')],
  -7.899856758621814),
 ([(0.0516545, 'gas'),
   (0.025557319, 'update'),
   (0.024599167, 'trading'),
   (0.018444642, '...'),
   (0.014154788, 'ena')],
  -10.453911658471792),
 ([(0.024499604, 'new'),
   (0.020149786, 'expense'),
   (0.019933272, 'ubs'),
   (0.017698081, 'management'),
   (0.017422402, 'document')],
  -11.376159617710819),
 ([(0.039823454, 'enron'),
   (0.029850056, 'request'),
   (0.025310017, 'netco'),
   (0.019665524, 'deal'),
   (0.017553488, 'tw')],
  -13.584480838697662),
 ([(0.08412206, 'meeting'),
   (0.021503825, 'plan'),
   (0.0182059, 'eol'),
   (0.012866424, 'today'),
   (0.012773377, 'enrononline')],
  -13.882475681546453)]

In [76]:
for index, score in sorted(lda_model_tfidf[exec_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, show_feature)))


Score: 0.5974438786506653	 
Topic: 0.016*"lunch" + 0.015*"update" + 0.014*"..." + 0.012*"gisb" + 0.011*"letter" + 0.010*"checkout" + 0.010*"ca"

Score: 0.10202427953481674	 
Topic: 0.016*"new" + 0.014*"enron" + 0.013*"credit" + 0.012*"california" + 0.011*"eol" + 0.011*"hi" + 0.010*"fwd"

Score: 0.100287526845932	 
Topic: 0.021*"agreement" + 0.016*"call" + 0.016*"contract" + 0.014*"hey" + 0.012*"conference" + 0.011*"tw" + 0.009*"contracts"

Score: 0.10021429508924484	 
Topic: 0.019*"power" + 0.015*"gas" + 0.015*"master" + 0.013*"energy" + 0.013*"agreement" + 0.012*"info" + 0.010*"project"

Score: 0.10003002732992172	 
Topic: 0.028*"deal" + 0.024*"meeting" + 0.012*"hello" + 0.009*"netco" + 0.009*"information" + 0.008*"enron" + 0.008*"isda"
